In [1]:
# This script takes work from Pytorch MNIST and UNet Dev.ipynb and makes training and validation repeatable 
# in order to test a method for saving and setting optimizer state (momentum, etc.) that remains consistent
# even when the restoring is done after restarting the kernel. The train_epoch method also has been reduced to training
# over a few batches (not a complete epoch) in order to make testing faster. Testing should be done for Unet
# with its many optimizers, and also for MNIST cnn with its one optimizer.
# .................. EDIT THIS .................................

In [1]:
import tfedlrn.collaborator.pytorchmodels

from tfedlrn.collaborator.pytorchmodels.pytorch2dunet import PyTorch2DUNet

Importing Collaborator


ImportError: cannot import name 'Collaborator'

In [9]:
sys.modules

{'IPython': <module 'IPython' from '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/__init__.py'>,
 'IPython.core': <module 'IPython.core' from '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/core/__init__.py'>,
 'IPython.core.alias': <module 'IPython.core.alias' from '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/core/alias.py'>,
 'IPython.core.application': <module 'IPython.core.application' from '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/core/application.py'>,
 'IPython.core.async_helpers': <module 'IPython.core.async_helpers' from '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/c

In [7]:
import sys
sys.path

['/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python35.zip',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/tfedlrn-0.0.0-py3.5.egg',
 '/home/edwardsb/.local/share/virtualenvs/spr_secure_intelligence-trusted_federated_-aNwh6wSd/lib/python3.5/site-packages/IPython/extensions',
 '/home/edwardsb/.ipython']

In [2]:
Collaborator()

NameError: name 'Collaborator' is not defined

In [3]:
Pytorch2DUNet()

NameError: name 'Pytorch2DUNet' is not defined

In [7]:
# Set here the type of optimizer that you want to test with

In [8]:
this_optimizer = 'Adam'  # NOTE: If model below is chosen as PyTorchMNISTCNN, SGD will be used instead !!!! 
this_model = PyTorchMNISTCNN  # PyTorch2DUNet  

In [9]:
device = torch.device("cuda")

In [10]:
#######################################################################################################

In [22]:
###############         EXPLORING SAVING AND RESTORING OPTIMIZER and MODEL          ###################

In [23]:
#######################################################################################################

In [24]:
cnn = this_model(device=device, optimizer=this_optimizer)

In [25]:
# Testing that the data used for training is reproducible by looking at its sum

In [26]:
cnn.train_partial_epoch()

(5.5569515, 'Sum of batch is data: -399206.46875, target: 86.0')

In [27]:
cnn.train_partial_epoch()

(5.556761, 'Sum of batch is data: -399206.46875, target: 86.0')

In [28]:
# testing that validation is reproducible for a given model

In [29]:
cnn.validate()

0.03455506780301221

In [30]:
cnn.validate()

0.03455506780301221

In [31]:
############################################################################################
# testing restoring model and optimizer, should result in training to the same validation ##
############################################################################################

In [32]:
initial_val = cnn.validate()
initial_val

0.03455506780301221

In [33]:
# save model
model_weights = cnn.get_tensor_dict()

In [34]:
# train for some more, then see that model is different now

In [35]:
cnn.train_partial_epoch()

(5.55657, 'Sum of batch is data: -399206.46875, target: 86.0')

In [36]:
# see valiation is not the same now
after_train_val = cnn.validate()
after_train_val

0.03455377297359519

In [37]:
# restore saved model
cnn.set_tensor_dict(model_weights)

In [38]:
# see validation is back to previously observed for saved model
initial_val == cnn.validate()

True

In [39]:
# see if grabbing the full state again, it gives the same thing as we had saved before

In [40]:
model_weights_2 = cnn.get_tensor_dict()

In [41]:
bool_array = np.array([])
for key in model_weights:
    if key.startswith('__opt_'):
        for inner_key in model_weights[key]:
            np.append(bool_array, model_weights[key][inner_key] == model_weights_2[key][inner_key])
    else:
        np.append(bool_array, [np.all(model_weights[key] == model_weights_2[key])])
np.all(bool_array)


True

In [42]:
# now train again and see you get back to the place you did before after running 'train_partial_epoch' once

In [43]:
cnn.train_partial_epoch()

(5.55657, 'Sum of batch is data: -399206.46875, target: 86.0')

In [44]:
after_train_val == cnn.validate()

True

In [46]:
#######################################

In [47]:
# ----now test that restoring can happen across processes

In [48]:
########################################

In [49]:
# saving model to disk as well as initial_val and after_train_val

In [50]:
import pickle
filenames = ['saved_model.pkl', 'initial_val.pkl', 'after_train_val.pkl']
objects = [model_weights, initial_val, after_train_val]
for ob, filename in zip(objects, filenames):
    with open(filename, 'wb') as file:
        pickle.dump(ob, file)

In [41]:
##############################################################################################################

In [42]:
######                         !!!!!!!!!!!!!!!!  RESTART KERNEL HERE !!!!!!!!!!!                 #############

In [10]:
######         (then run the top cells of this workbook up to device intialization)            ###############

In [11]:
#############################################################################################################

In [11]:
###### Then return to this point and run cells below ####################

In [12]:
#initiate model and train a bit, then restore model from disk

In [13]:
cnn = this_model(device, optimizer=this_optimizer)

In [14]:
cnn.train_partial_epoch()

(5.514941, 'Sum of batch is data: -399206.46875, target: 86.0')

In [15]:
# get model weights and validation values from disk

In [16]:
filenames = ['saved_model.pkl', 'initial_val.pkl', 'after_train_val.pkl']
object_names = ['model_weights', 'initial_val', 'after_train_val']
for object_name, filename in zip(object_names, filenames):
    with open(filename, 'rb') as file:
        vars()[object_name] = pickle.load(file)



In [17]:
# see valiation is not the same as the ones before
this_val = cnn.validate()
initial_val != this_val and after_train_val != this_val 

True

In [18]:
# restore model to saved values
cnn.set_tensor_dict(model_weights)

In [19]:
# see validation is back to previously observed for saved model
initial_val == cnn.validate()

True

In [20]:
# see if grabbing the full state again, it gives the same thing as we had saved before

In [21]:
model_weights_2 = cnn.get_tensor_dict()

In [22]:
bool_array = np.array([])
for key in model_weights:
    if key.startswith('__opt_'):
        for inner_key in model_weights[key]:
            np.append(bool_array, model_weights[key][inner_key] == model_weights_2[key][inner_key])
    else:
        np.append(bool_array, [np.all(model_weights[key] == model_weights_2[key])])
np.all(bool_array)


True

In [23]:
# now train to get a new validation, and see that it matches what we had after training once before
cnn.train_partial_epoch()

(5.55657, 'Sum of batch is data: -399206.46875, target: 86.0')

In [24]:

after_train_val == cnn.validate()

True